In [ ]:
dataset = [
    {
        "question": "Qu’est-ce qu’une variable en programmation ?",
        "answer": "C’est un espace mémoire nommé qui permet de stocker une valeur.",
    },
    {
        "question": "Quelle est la différence entre une liste et un tableau ?",
        "answer": "Une liste est dynamique alors qu’un tableau a une taille fixe.",
    },
    {
        "question": "À quoi sert une boucle for ?",
        "answer": "Elle répète un bloc d’instructions un nombre déterminé de fois.",
    },
    {
        "question": "Qu’est-ce qu’une fonction ?",
        "answer": "Un bloc de code réutilisable prenant des paramètres et pouvant retourner une valeur.",
    },
    {
        "question": "Qu’est-ce qu’une API ?",
        "answer": "Une interface permettant à des programmes de communiquer entre eux.",
    },
    {
        "question": "Que signifie debugger un programme ?",
        "answer": "Trouver et corriger les erreurs dans le code.",
    },
    {
        "question": "Qu’est-ce qu’une exception ?",
        "answer": "Une erreur détectée à l'exécution qui peut interrompre le programme.",
    },
    {
        "question": "Différence entre == et === en JavaScript ?",
        "answer": "== compare les valeurs avec conversion de type, === compare strictement type et valeur.",
    },
    {
        "question": "Qu’est-ce qu’un algorithme ?",
        "answer": "Une suite d’étapes permettant de résoudre un problème.",
    },
    {
        "question": "Que signifie open source ?",
        "answer": "Code source librement accessible et modifiable.",
    },
    {
        "question": "Qu’est-ce qu’un objet en POO ?",
        "answer": "Une instance d'une classe avec attributs et méthodes.",
    },
    {
        "question": "À quoi sert Git ?",
        "answer": "À versionner le code et suivre les modifications.",
    },
    {
        "question": "Qu’est-ce qu’une boucle infinie ?",
        "answer": "Une boucle qui ne s’arrête jamais car sa condition reste vraie.",
    },
    {
        "question": "Qu’est-ce qu’une requête SQL SELECT ?",
        "answer": "Elle permet de lire des données dans une base de données.",
    },
    {
        "question": "Que signifie frontend et backend ?",
        "answer": "Frontend : interface utilisateur. Backend : logique serveur et gestion des données.",
    },
]

In [ ]:
import random


def get_random_question_answer(nb_questions=1):

    assert nb_questions <= len(dataset), "nb_questions exceeds dataset size"
    assert nb_questions > 0, "nb_questions must be at least 1"

    return random.sample(dataset, nb_questions)